# Brain Tumor MRI Classification: Validation + XAI (Notebook)
This notebook runs the released pipeline end-to-end using the repo scripts:
- Create stratified 64/16/20 splits
- Extract features (MobileNetV2 + EfficientNetV2B0, GAP+concat)
- Train KNN (k=5, Euclidean, distance weights)
- Evaluate (confusion, ROC)
- Generate XAI (Grad-CAM + SHAP) via one-shot script

Data layout required:
data/images/{glioma,meningioma,pituitary,notumor}

In [ ]:
import sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())

In [ ]:
!python scripts/make_splits_nosklearn.py

In [ ]:
!python src/pipeline/extract_features.py --config configs/default.yaml
!python src/pipeline/train_knn.py --config configs/default.yaml
!python src/pipeline/evaluate.py --config configs/default.yaml

In [ ]:
!python scripts/xai_all_in_one.py

In [ ]:
import os
to_check = ['results/test/confusion.png','results/test/roc_curves.png',
           'outputs/xai/gradcam','outputs/xai/shap']
for p in to_check:
    print(p, 'OK' if os.path.exists(p) else 'MISSING')